## 管理层信息抽取

In [ ]:
! cp /Users/alchemy_taotaox/Desktop/next/my_test/* ../corpus/

In [ ]:
! ls ../corpus

## 算法基本思路
1. 扫描pdf 获得页眉页脚
2. 扫描pdf 获得最小和最大的x坐标
    如果一个字符是最小的坐标，那么就和之前的行合并为一个段落
    如果一个字符是最大的坐标，那么就和之前的
2. 提取文档中的表格和字符串。

In [83]:
from pdfplumber.pdf import PDF
import re
from collections import Counter

def is_header(str, header):
    if header is None:
        return False
    return str == header

def is_foot(str, foot):
    if foot is None:
        return False
    
    s = re.sub(r'\s*\d+\s*', '#num#', str)
    return s == foot or s == '#num#/#num#'

def get_foot_header(pdf:PDF, check_pages_num=20, threshold = 0.9):
    head_counter = Counter()
    foot_counter = Counter()
    pages_num = len(pdf.pages)
    if pages_num < check_pages_num:
        check_pages_num = pages_num
    for i in range(check_pages_num):
        text_lines = pdf.pages[i].extract_words()
        if text_lines is not None and len(text_lines) == 0:
            continue
        top_line = text_lines[0]['text']
        bottom_line = text_lines[-1]['text']
        # print(bottom_line, " !!!!! ")
        bottom_line = re.sub(r'\d+', '#num#', bottom_line)
        head_counter.update([top_line])
        foot_counter.update([bottom_line])
    head, foot = "fadfasdfa", "dfadfadsfsdf"
    try:    
        most_common_head_word, count_head = head_counter.most_common(1)[0]
        most_common_foot_word, count_foot = foot_counter.most_common(1)[0]

   
        if count_head *1./check_pages_num >= threshold:
            head = most_common_head_word
        if count_foot *1./check_pages_num >= threshold:
            foot = most_common_foot_word
    except:
        pass
    return head, foot
    
    

In [44]:
import pdfplumber

In [45]:
from operator import itemgetter
import json

def check_bboxes(word, table_bbox):
    """
    Check whether word is inside a table bbox.
    """
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]



def get_lines_from_page(page, head, foot):
    tables = page.find_tables()
    table_bboxes = [i.bbox for i in tables]
    tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
    non_table_words = [word for word in page.extract_words() if not any(
        [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
    lines = []
    lines_x0_x1 =[]
    for cluster in pdfplumber.utils.cluster_objects(
        non_table_words + tables, itemgetter('top'), tolerance=5):
        if 'text' in cluster[0]:
            x_0 = x_1 = -1
            inner_lines = []
            for item in cluster:
                #if len(cluster) == 1 and (is_foot(item['text'], foot) or  is_header(item['text'], head)):
                #    continue
                # print(item, cluster)
                if x_0 == -1:
                    if 'x0' in item:
                        x_0 = item['x0']
                if 'x1' in item:
                    x_1 = item['x1']
                if 'text' in item:
                    inner_lines.append(item['text'])
                elif 'table' in item:
                    inner_lines.append("\n" + json.dumps(item['table'], ensure_ascii=False) + "\n")
            if len(inner_lines) > 0:
                lines.append(' '.join(inner_lines))
                lines_x0_x1.append((x_0, x_1))
                
            # lines.append(' '.join([i['text'] for i in cluster if not is_foot(i['text'], foot) and not is_header(i['text'], head)]))
        elif 'table' in cluster[0]:
            lines.append(cluster[0]['table'])
            lines_x0_x1.append((10000000, -1))

    # Find the minimum of the first elements
    if len(lines_x0_x1) == 0:
        return []
    min_first = min(x[0] for x in lines_x0_x1)

    # Find the maximum of the second elements
    max_second = max(x[1] for x in lines_x0_x1)
       
    # print(lines)
    
    # lines = lines[1:-1]
    if type(lines[0]) == str and is_header(lines[0], head):
        lines = lines[1:]
    if type(lines[-1]) == str and is_foot(lines[-1], foot):
        lines = lines[:-1]
    for index, item in enumerate(lines):
        if type(lines[index]) == str:
            lines[index] = lines[index].strip()
        
        # 如果锁进， 那么要换行
        
        if abs(lines_x0_x1[index][0] - min_first) > 0.1 and lines_x0_x1[index][0] < 1000000:
            # print(lines_x0_x1[index][0], min_first, "----")
            if type(lines[index]) == str:
                lines[index] = "\n" + lines[index]
        # 如果没有写完，那么换行
        if abs(lines_x0_x1[index][1] - max_second) > 16 and lines_x0_x1[index][1] > 0:
            # print(lines_x0_x1[index][1], max_second, "----", lines[index])
            if type(lines[index]) == str:
                lines[index] = lines[index] + "\n"
        elif lines_x0_x1[index][1] > 0:
            # print('******FULL LINE******', lines_x0_x1[index][1], max_second, "----", lines[index])
            pass
        # print(lines[index], end='')
    return lines

In [76]:
filename = "../corpus/year_report_2.pdf"
filename = "/tmp/b.PDF"
# filename = "/tmp/c.PDF"
# filename = "/tmp/e.PDF"
filename = "/tmp/f.PDF"
pdf = pdfplumber.open(filename)
# page = pdf.pages[10]
# page = pdf.pages[15]
page = pdf.pages[7]
# print(page.extract_text())
head, foot = get_foot_header(pdf)
print(head)
print(foot)
lines = get_lines_from_page(page, head, foot)
for line in lines:
    print("----", line, "----", type(line))

# print(pdf.pages[7].extract_words() )# 计算一个字符大概的宽度是多少，然后定义magic distance = 16
# print(pdf.pages[7].extract_text())

苏州天沃科技股份有限公司2022年年度报告全文
#num#
---- 
一、重大环保问题 ............................................................................................................................................................... 58 ---- <class 'str'>
---- 
二、社会责任情况 ............................................................................................................................................................... 58 ---- <class 'str'>
---- 
三、巩固拓展脱贫攻坚成果、乡村振兴的情况 ........................................................................................................ 58 ---- <class 'str'>
---- 
第六节 重要事项 ............................................................................................................................................................................ 60 ---- <class 'str'>
---- 一、承诺事项履行情况 ...................................................................................................................................................... 60 ---- <class 'str'>
---- 
二、控股股东及其他关联方对上市公司

In [77]:
from pdfplumber.pdf import PDF
import json
def begin_mda(lines):
    len_lines = len(lines)
    if len_lines < 1:
        return False, -1
    # if type(lines[0]) != str:
    #     return False, -1
    # print(" check beign count ", len_lines)
    for index, item in enumerate(lines):
        # print("CHECK", item, index)
        if type(item) != str:
            continue
        if item.find('管理层讨论与分析') != -1 and item.find('第三节')!= -1 and item.find("...............") == -1 and item.find("请") == -1:
            # print(' BEGIN ', index)
            return True, index
    return False, -1
def end_mda(lines):
    if len(lines) < 1:
        return False, -1
    # if type(lines[0]) != str:
    #     return False, -1
    # print("check: ", lines[0], "$$$$$$")
    for index, item in enumerate(lines):
        if type(item) != str:
            continue
        if item.find('公司治理') != -1 and item.find('第四节')!= -1 and item.find("...............") == -1 and item.find("请") == -1:
            return True, index
    return False, -1

def get_all_lines_about_mda(pdf:PDF, head:str, foot:str):
    begin = False
    end = False
    all_lines = []
    index = 0
    # print(f"page number: {len(pdf.pages)}")
    for page in pdf.pages:
        index += 1
        if index < 5:
            continue
        # print(index, " ----- page ", head, foot)
        lines = get_lines_from_page(page, head, foot)
        # for line in lines:
        #     print(line)
        begin_mda_status, line_index = begin_mda(lines)
        # print(index, ' ---page   check ------', begin_mda_status)

        if begin is False and begin_mda_status:
            begin = True
            lines = lines[line_index:]
        end_mda_status, line_index = end_mda(lines)
        if begin and end_mda_status:
            
            all_lines.append(lines[:line_index])
            break
        if begin is True:
            all_lines.append(lines)
    # print(f" final page index {index}")
    content_list = []
    for lines in all_lines:
        for line in lines:
            if type(line) != str:
                line = json.dumps(line, ensure_ascii=False)
            # print(line, end='')
            content_list.append(line)
    return "".join(content_list)
            
content = get_all_lines_about_mda(pdf, head, foot)
if content is not None:
    print(content)
else:
    print("None")

第三节 管理层讨论与分析

一、报告期内公司所处行业情况
公司需遵守《深圳证券交易所上市公司自律监管指引第3号——行业信息披露》中土木工程建筑业的披露要求
1．能源工程行业

就新能源领域，国务院于2021年10月印发了《2030年前碳达峰行动方案》，包括推进煤炭消费替代
和转型升级、大力发展新能源、加快建设新型电力系统等主要目标，推动我国能源结构向清洁低碳安全高效转变。新型电力系统是在安全的前提下，服务于新能源快速增长目标而建设的系统性工程，与之对应的体系化变革将带来产业链系统性的投资机遇。在此背景下，以风电、光伏发电为主的新能源装机持续高速增长，天沃科技所从事的新能源工程设计、勘察和 EPC 总承包业务的发展前景也较为乐观。但是，随着新能源补贴政策调整、新能源组件价格波动以及市场竞争加剧，新能源工程领域业务风险也有所上升。就传统火电领域，中央经济工作会议指出，“要立足以煤为主的基本国情，抓好煤炭清洁高效利用，

增加新能源消纳能力，推动煤炭和新能源优化组合”。近年来，受煤炭和燃气价格高涨影响，火力发电企业经营效益普遍不佳，对存量机组改造意愿的不强。但从长远考虑，为保证新型电力系统安全性，传统煤电装备灵活性改造需求将逐步提升。此外，受近期电力供需结构性偏紧趋势影响，为保障用电安全，在政策推动下，大规格的煤电项目投资建设呈复苏态势，中小规格的煤电项目启动则存在不确定性。总体而言，对于新型电力系统中传统能源及新能源的发展，我国形成了既要重视新能源快速增长，

又要重视传统能源在电力系统的重要保供作用的方针。新能源及传统能源均将在能源系统变革下迎来新的发展机遇。2．高端压力容器行业

就化工装备领域，中央经济工作会议指出“新增可再生能源和原料用能不纳入能源消费总量控制”，原

料煤不在“双碳目标”约束范围内，从而有望推动煤化工产业转型升级，加快淘汰落后产能。在“十四五”期间，我国将重点开展煤制油、煤制天然气、低阶煤分质利用、煤制化学品、煤炭和石油综合利用等五个领域以及通用技术装备的升级示范，推动煤炭深加工产业向更高水平发展。此外，基于我国“富煤贫油少气”的能源禀赋，发展现代煤化工是国家能源发展战略的重要组成部分，是充分发挥煤炭能源相对资源优势，保障国家能源安全的必要措施，是缓解石油和天然气供需矛盾的现实手段。煤资源的高效利用将推动煤化工行业向高端化、多元化、低碳化方向发

## 抓取信息



In [ ]:
from datetime import datetime, timedelta

def split_interval(start, end, day_cnt):
    date_format = "%Y-%m-%d"
    start_date = datetime.strptime(start, date_format)
    end_date = datetime.strptime(end, date_format)
    
    result = []

    while start_date < end_date:
        # Calculate the end date of the current interval.
        interval_end_date = start_date + timedelta(days=day_cnt-1)

        # Ensure that the interval doesn't go beyond the overall end date.
        if interval_end_date > end_date:
            interval_end_date = end_date

        # Append the current interval to the result list.
        result.append(
            (start_date.strftime(date_format), 
             interval_end_date.strftime(date_format), 
             (interval_end_date - start_date).days + 1)
        )

        # Move the start date to the next day after the current interval.
        start_date = interval_end_date + timedelta(days=1)

    return result
results = split_interval('2023-01-01', '2023-07-31', 1)

print(results)

In [ ]:
import requests
import time
from tqdm import tqdm
import json




def request_data(page_size=1000, page_num=1, se_date="2023-04-26~2023-04-26"):
    url = "http://www.cninfo.com.cn/new/hisAnnouncement/query"
    if page_num > 1:
        url = "http://www.cninfo.com.cn/new/hisAnnouncement/query"

    params = {
        "pageNum": page_num,
        "pageSize": page_size,
        "column": "szse",
        "tabName": "fulltext",
        "plate": "",
        "stock": "",
        "searchkey": "",
        "secid": "",
        "category": "category_ndbg_szsh",
        "trade": "",
        "seDate": se_date,
        "sortName": "",
        "sortType": "",
        "isHLtitle": "true"
    }

    response = requests.post(url, data=params)

    if response.status_code == 200:
        return response.json()  # 或者返回response.text取决于你需要什么样的数据格式
    else:
        return None

file = open('report_meta_info.txt', 'w')
results = split_interval('2023-01-01', '2023-07-31', 1)
results = results[::-1]
#for date_range in results:
for i, date_range in enumerate(tqdm(results)):
    se_date = f"{date_range[0]}~{date_range[1]}"
    
    page_size = 30
    data = request_data(page_size=10, se_date=se_date)
    try_times = 0
    while data is None:
        time.sleep(1)
        data = request_data(page_size=10, se_date=se_date)
        try_times += 1
        print(f"craw data wrong, try {try_times}")
        if try_times == 3:
            break  
    if data is None:
        continue
    page_count = (data['totalAnnouncement'] -1 )// page_size  + 1
    print(page_count)
    begin = time.time()
    for index in range(1, page_count+1):
    
        data = request_data(page_num=index, page_size=page_size, se_date=se_date)
        try_times = 0
        while data is None:
        
            time.sleep(3)
            data = request_data(page_num=index, page_size=page_size, se_date=se_date)
            try_times += 1
            print(f"craw data wrong, try {try_times}")
            if try_times == 3:
                break
        
        last = time.time() - begin
        need_time = last/index * page_count  - last
        every_time = last*1./index
        print(f"{index}/{page_count}: last time {last} 秒， need {need_time}, every page cost: {every_time}, count: { len(data['announcements'])}, but page size is {page_size}")
        for item in data['announcements']:
            file.write(json.dumps(item, ensure_ascii=False) + "\n")
file.close()
    

In [84]:
from urllib.parse import urljoin
import os
from tqdm import tqdm
import requests
import json

def download_and_process_file(url, local_path='/tmp/a.PDF', try_count=3):
    try_times = 0
    response = None
    while True:
        # 下载文件
        response = requests.get(url, proxies={})
        if response.status_code == 200:
            break
            # return response.json()  # 或者返回response.text取决于你需要什么样的数据格式
        else:
            response = None
        try_times += 1
        if try_times >= try_count:
            time.sleep(3)
            break
    if response is None:
        return response
    # response.raise_for_status()  # 确保请求成功

    # 将文件写入本地
    with open(local_path, 'wb') as output_file:
        output_file.write(response.content)

    # 读取PDF文件的内容
    
    pdf = pdfplumber.open(local_path)
    head, foot = get_foot_header(pdf)
    content = get_all_lines_about_mda(pdf, head, foot)
    #print(content)
    # 删除临时文件
    os.remove(local_path)

    return content
'''
1. read meta file
2. download pdf file into tmp file
3. get the necessary content
4. write data into file
   {'stock_code', 'stock_name', 'content':   'url':  'pubtime': xxx}

'''
import os
if 'all_proxy' in os.environ:
    print("pop all proxy")
    os.environ.pop('all_proxy')
base_url = 'http://static.cninfo.com.cn/'
meta_file = 'report_meta_info.txt'
data_file = 'report_data_info.txt'
file = open(data_file, "w")
lines = open(meta_file).readlines()
code_set = set()
for index, line in tqdm(enumerate(lines)):
    # if index < 60:
    #     continue
    info = json.loads(line.strip())
    '''
    "secCode": "001211", "secName": "双枪科技, adjunctUrl
    '''
    title = info['announcementTitle']
    if title.find('关于') != -1 or title.find("摘要") != -1 or title.find('2022') == -1:
        continue
    stock_code = info['secCode']
    if stock_code in code_set:
        continue
    # code_set.add(stock_code)
    stock_name = info['secName']
    relative_url = info['adjunctUrl']
    pub_time = relative_url.split('/')[-2]
    url =  urljoin(base_url, relative_url)
    content = download_and_process_file(url, "/tmp/a.PDF")
    if content is None or content == "":
        pass
    else:
        code_set.add(stock_code)
    result = {"title": title, 'pubtime': pub_time, 'stock_code': stock_code, 'stock_name': stock_name, 'content': content, 'url': url}
    file.write(json.dumps(result, ensure_ascii=False) + "\n")
    # if index == 100:
    #     break

   
   

100it [04:24,  2.64s/it]
